# create database from scratch

In [2]:
from flask import Flask 
from flask_sqlalchemy import SQLAlchemy
from faker import Faker
from datetime import datetime, timedelta 
import os 

# filename is: ./database.db
base_dir = os.getcwd()
db_path = os.path.join(base_dir, 'database.db')

# initialize
faker = Faker('en_AU')
db = SQLAlchemy()
app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + db_path
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db.init_app(app)

In [3]:
class Student(db.Model):
    __tablename__ = 'student'
    __table_args__ = {"extend_existing": True}

    student_id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String, nullable=False, unique=True)
    first_name = db.Column(db.String, nullable=False)
    last_name = db.Column(db.String, nullable=False)
    email = db.Column(db.String, nullable=False, unique=True)
    password = db.Column(db.String, nullable=False)
    avatar = db.Column(db.String, nullable=False)
    
    qualification = db.Column(db.String, nullable=False)
    school_name = db.Column(db.String, nullable=False)
    major = db.Column(db.String, nullable=False)
    skills = db.Column(db.String, nullable=False)
    strength = db.Column(db.String, nullable=False)
    
    # !!!!    
    resume_url = db.Column(db.String, nullable=False, unique=True)


class Supervisor(db.Model):
    __tablename__ = 'supervisor'
    __table_args__ = {"extend_existing": True}

    supervisor_id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String, nullable=False, unique=True)
    first_name = db.Column(db.String, nullable=False)
    last_name = db.Column(db.String, nullable=False)
    email = db.Column(db.String, nullable=False, unique=True)
    password = db.Column(db.String, nullable=False)
    avatar = db.Column(db.String, nullable=False)
    
    qualification = db.Column(db.String, nullable=False)
    school_name = db.Column(db.String, nullable=False)
    skills = db.Column(db.String, nullable=False)
    

class Partner(db.Model):
    __tablename__ = 'partner'
    __table_args__ = {"extend_existing": True}

    partner_id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String, nullable=False, unique=True)
    first_name = db.Column(db.String, nullable=False)
    last_name = db.Column(db.String, nullable=False)
    email = db.Column(db.String, nullable=False, unique=True)
    password = db.Column(db.String, nullable=False)
    avatar = db.Column(db.String, nullable=False)
    
    company = db.Column(db.String, nullable=False)
    position = db.Column(db.String, nullable=False)
    description = db.Column(db.String, nullable=False)


# the partner creates the project
class Project(db.Model):
    __tablename__ = 'project'
    __table_args__ = {"extend_existing": True}

    # each partner may have multiple project
    project_id = db.Column(db.Integer, primary_key=True)
    partner_id = db.Column(db.Integer, db.ForeignKey('partner.partner_id'), nullable=False)
    
    title = db.Column(db.String, nullable=False)
    description = db.Column(db.String, nullable=False)

    start_date = db.Column(db.DateTime, nullable=False)
    end_date = db.Column(db.DateTime, nullable=False)
    num_of_days = db.Column(db.Integer, nullable=False)

    # the project detail
    problem_statement = db.Column(db.String, nullable=False)
    desired_outcomes = db.Column(db.String, nullable=False)
    required_skills = db.Column(db.String, nullable=False)
    deliverables = db.Column(db.String, nullable=False)

    # the requirements for students
    requirements = db.Column(db.String, nullable=False)
        

# the student express interests, and attach a sentence
class InterestExpress(db.Model):
    __tablename__ = 'interest_express'
    __table_args__ = {"extend_existing": True}
    
    # each student may have multiple interests to multiple projects
    interest_express_id = db.Column(db.Integer, primary_key=True)
    student_id = db.Column(db.Integer, db.ForeignKey('student.student_id'), nullable=False)
    project_id = db.Column(db.Integer, db.ForeignKey('project.project_id'), nullable=False)
    reason = db.Column(db.String, nullable=False)
    last_updated_at = db.Column(db.DateTime, nullable=False, default=datetime.now, onupdate=datetime.now)


# the supervisor finally assign the student to the project
class ProjectMember(db.Model):
    __tablename__ = 'project_member'
    __table_args__ = {"extend_existing": True}
    
    # each project contain multiple students, assign by the supervisor, with a reason
    project_member_id = db.Column(db.Integer, primary_key=True)
    project_id = db.Column(db.Integer, db.ForeignKey('project.project_id'), nullable=False)
    student_id = db.Column(db.Integer, db.ForeignKey('student.student_id'), nullable=False)
    supervisor_id = db.Column(db.Integer, db.ForeignKey('supervisor.supervisor_id'), nullable=False)
    assign_reason = db.Column(db.String, nullable=False)
    
    # final feedback put into here???
    final_supervisor_feedback = db.Column(db.String, nullable=False)
    final_supervisor_mark = db.Column(db.Integer, nullable=False)
    final_supervisor_last_updated_at = db.Column(db.DateTime, nullable=False, default=datetime.now, onupdate=datetime.now)
    
    final_partner_feedback = db.Column(db.String, nullable=False)
    final_partner_mark = db.Column(db.Integer, nullable=False)
    final_partner_last_updated_at = db.Column(db.DateTime, nullable=False, default=datetime.now, onupdate=datetime.now)
    
    final_student_feedback = db.Column(db.String, nullable=False)
    final_student_last_updated_at = db.Column(db.DateTime, nullable=False, default=datetime.now, onupdate=datetime.now)
    
    
# the student reports for progress for the project, 
# recorded with date, and content, and marked by the supervisor and partner
class ProjectProgress(db.Model):
    __tablename__ = 'project_progress'
    __table_args__ = {"extend_existing": True}
    
    # each student may have multiple progress for a project
    project_progress_id = db.Column(db.Integer, primary_key=True)
    project_id = db.Column(db.Integer, db.ForeignKey('project.project_id'), nullable=False)
    student_id = db.Column(db.Integer, db.ForeignKey('student.student_id'), nullable=False)
    
    content = db.Column(db.String, nullable=False)
    student_last_updated_at = db.Column(db.DateTime, nullable=False, default=datetime.now, onupdate=datetime.now)
    
    supervisor_id = db.Column(db.Integer, db.ForeignKey('supervisor.supervisor_id'), nullable=False)
    supervisor_mark = db.Column(db.Integer, nullable=False)
    supervisor_feedback = db.Column(db.String, nullable=False)
    supervisor_last_updated_at = db.Column(db.DateTime, nullable=False, default=datetime.now, onupdate=datetime.now)
    
    partner_id = db.Column(db.Integer, db.ForeignKey('partner.partner_id'), nullable=False)
    partner_mark = db.Column(db.Integer, nullable=False)
    partner_feedback = db.Column(db.String, nullable=False)
    partner_last_updated_at = db.Column(db.DateTime, nullable=False, default=datetime.now, onupdate=datetime.now)


In [4]:
with app.app_context():
    db.drop_all()
    db.create_all()